In [1]:
import time
import requests, re
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests
from urllib import request
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from IPython.core.display import display, HTML
from selenium.common.exceptions import UnexpectedAlertPresentException

In [1]:
def jk_url_crawler(keyword,page_num):
    url_list = []
    date_list = []
    for page in range(page_num+1):
        url = f'https://www.jobkorea.co.kr/User/Qstn/TalkSearchWindow?Keyword={keyword}&OrderType=1&W_TypeCode=0&page={page}'
        driver = webdriver.Chrome()
        driver.get(url)
        html_source = driver.page_source
        soup = BeautifulSoup(html_source, 'lxml')
        content_area = soup.find_all("div",class_='contArea')
        author_area = soup.find_all('div',class_='post-cell-box')
        for i in content_area:
            href = i.find_all('a')
            for h in href:
                url = h.attrs['href']
                url = 'https://www.jobkorea.co.kr/' + url
                print(page)
                print(url)
                url_list.append(url)
    
        for i in author_area:
            date = i.find('cell',class_='span')
            print(date.text)
            date_list.append(date.text)
                
        time.sleep(3)
        driver.close()
    find_s = '#'
    drop_url = [i for i in range(len(url_list)) if find_s in url_list[i]]
    url_list = np.delete(url_list, drop_url, axis=0)

    return url_list, date_list

keyword = "산업공학과"

In [63]:
def jobKorea_crawler(new_url, new_date):
    final_content = pd.DataFrame(columns = ['title','content','date'])
    final_comment = pd.DataFrame(columns = ['comment','date'])

    for i in range(len(new_url)):
        print(i)
        driver = webdriver.Chrome()
        driver.get(new_url[i])
        res = requests.get(new_url[i])
        soup = BeautifulSoup(res.text, "lxml")
        comment_list = []
        title_list = []
        content_list = []
        date_content = []
        date_comment = []
        df1 = pd.DataFrame()
        df2 = pd.DataFrame()
        try : 
            title = soup.find('p',class_='tit').text
            title_list.append(title)
            content = soup.find('div',class_='cont').text
            content_list.append(content)
            date_content.append(new_date[i])
            comment_area = soup.find_all('p',class_='cont')    
            comment_list = []
            for j in comment_area:
                com = re.sub('(<([^>]+)>)', '', str(j))
                comment_list.append(com)
                date_comment.append(new_date[i])
        except AttributeError:
            pass

        except UnexpectedAlertPresentException as e:
            print(e.__dict__["msg"])
            
        
        df1 = pd.DataFrame({'title':title_list,'content':content_list,'date':date_content})
        df2 = pd.DataFrame({'comment':comment_list, 'date':date_comment})
        final_content = pd.concat([final_content, df1])
        final_comment = pd.concat([final_comment, df2])
        driver.close()
        time.sleep(3)
    return final_content, final_comment